In [ ]:
import os
import json
import re
import subprocess
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

from system_info import retrieve_system_info

# Fix: gradio 6.x uses click.Choice[str] subscript at import-time, which
# click 8.x doesn't support on Python 3.13.  Patch before importing gradio.
import click
if not hasattr(click.Choice, "__class_getitem__"):
    click.Choice.__class_getitem__ = classmethod(lambda cls, _: cls)

import gradio as gr

load_dotenv(override=True)

openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

# Single OpenAI-compatible client pointing at OpenRouter (same as day3)
llm_client = OpenAI(
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1",
)

OPENAI_MODEL = "gpt-5.2"
CLAUDE_MODEL = "anthropic/claude-haiku-4-5"
GEMINI_MODEL = "google/gemini-2.5-flash-lite"

print("Setup complete.")

In [32]:
# ── System prompt used internally when converting Python → C++ ────────────────
_CPP_SYSTEM_PROMPT = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""


# ── Tool 1: convert Python to C++ via a dedicated LLM call ───────────────────

def convert_to_cpp(python_code: str) -> str:
    """Call the LLM to port Python code to high-performance C++.

    Mirrors the port() + user_prompt_for() pattern from day3.
    Returns the raw C++ source as a string (markdown fences stripped).
    """
    # Embed current system info so the LLM picks the right flags/features
    system_info = retrieve_system_info()

    user_prompt = f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}

Your response will be written to a file called main.cpp and then compiled and executed.
Respond ONLY with C++ code — no markdown, no explanation.

Python code to port:
```python
{python_code.strip()}
```
"""
    # Use reasoning_effort="high" for GPT models (same as port() in day3)
    reasoning_effort = "high" if "gpt" in OPENAI_MODEL else None
    try:
        resp = llm_client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": _CPP_SYSTEM_PROMPT},
                {"role": "user",   "content": user_prompt},
            ],
            reasoning_effort=reasoning_effort,
        )
    except TypeError:
        # Some OpenRouter backends reject unknown kwargs — fall back gracefully
        resp = llm_client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": _CPP_SYSTEM_PROMPT},
                {"role": "user",   "content": user_prompt},
            ],
        )

    cpp = resp.choices[0].message.content or ""
    # Strip markdown code fences the model may wrap the output in
    cpp = re.sub(r"^```(?:cpp|c\+\+)?\s*\n", "", cpp, flags=re.IGNORECASE)
    cpp = re.sub(r"\n```\s*$", "", cpp)
    return cpp.strip()


# ── Tool 2: save C++ source code to a file ────────────────────────────────────

def save_cpp_to_file(cpp_code: str, filename: str = "main.cpp") -> str:
    """Write C++ source code to disk.

    Mirrors write_output() from day3.
    Returns the absolute path of the saved file so the agent can report it.
    """
    path = Path(filename).resolve()
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(cpp_code, encoding="utf-8")
    return str(path)


# ── Tool 3: gather system info to derive compile/run commands ─────────────────

def get_system_info() -> str:
    """Return system environment details as a JSON string.

    The agent uses this to decide the right g++ flags and run command
    for the current machine (OS, CPU SIMD support, compiler version, etc.).
    Calls retrieve_system_info() which is already imported from system_info.py.
    """
    info = retrieve_system_info()
    return json.dumps(info, indent=2)


# ── Tool 4: compile the C++ file and run it ───────────────────────────────────

def compile_and_run(compile_command: str, run_command: str) -> str:
    """Compile and execute a C++ source file.

    compile_command : JSON array string, e.g.
        '["g++", "-std=c++20", "-Ofast", "-march=native", "-flto", "-DNDEBUG", "main.cpp", "-o", "main"]'
    run_command     : JSON array string, e.g.  '["./main"]'

    Mirrors compile_command / run_command from day3, but accepts them as
    JSON strings so the LLM can pass whatever the system requires.
    Returns the program's stdout, or a descriptive error if it fails.
    """
    try:
        compile_cmd = json.loads(compile_command)
        run_cmd     = json.loads(run_command)
    except json.JSONDecodeError as exc:
        return f"Could not parse command JSON: {exc}"

    # ── compile ──────────────────────────────────────────────────────────────
    compile_result = subprocess.run(compile_cmd, text=True, capture_output=True)
    if compile_result.returncode != 0:
        return (
            "Compilation failed.\n"
            f"stderr:\n{compile_result.stderr or compile_result.stdout}"
        )

    # ── run ──────────────────────────────────────────────────────────────────
    run_result = subprocess.run(run_cmd, text=True, capture_output=True)
    if run_result.returncode != 0:
        return (
            f"Compilation succeeded but execution failed (exit {run_result.returncode}).\n"
            f"stderr:\n{run_result.stderr}"
        )

    output = run_result.stdout.strip()
    return output if output else "Program ran successfully and produced no output."


In [25]:
# ── Tool schemas (OpenAI function-calling format) ─────────────────────────────

convert_to_cpp_function = {
    "name": "convert_to_cpp",
    "description": (
        "Convert Python source code to high-performance C++. "
        "Returns the full C++ source as a string."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "python_code": {
                "type": "string",
                "description": "The complete Python source code to convert.",
            }
        },
        "required": ["python_code"],
        "additionalProperties": False,
    },
}

save_cpp_to_file_function = {
    "name": "save_cpp_to_file",
    "description": (
        "Save C++ source code to a file on disk. "
        "Returns the absolute file path where it was saved."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "cpp_code": {
                "type": "string",
                "description": "The C++ source code to write to disk.",
            },
            "filename": {
                "type": "string",
                "description": "Destination filename (default: main.cpp).",
            },
        },
        "required": ["cpp_code"],
        "additionalProperties": False,
    },
}

get_system_info_function = {
    "name": "get_system_info",
    "description": (
        "Retrieve OS, CPU, and compiler details for this machine. "
        "Use the returned information to choose the correct g++ flags "
        "and run command before compiling C++ code."
    ),
    "parameters": {
        "type": "object",
        "properties": {},
        "additionalProperties": False,
    },
}

compile_and_run_function = {
    "name": "compile_and_run",
    "description": (
        "Compile a C++ source file and execute the resulting binary. "
        "Returns the program's stdout output, or an error message."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "compile_command": {
                "type": "string",
                "description": "JSON array string for the compile step, e.g. '[\"g++\", \"-std=c++20\", \"-Ofast\", \"-march=native\", \"-flto\", \"-DNDEBUG\", \"main.cpp\", \"-o\", \"main\"]'",
            },
            "run_command": {
                "type": "string",
                "description": "JSON array string for the run step, e.g. '[\"./main\"]'",
            },
        },
        "required": ["compile_command", "run_command"],
        "additionalProperties": False,
    },
}

# Collect all tools in the same list structure as the example flow
tools = [
    {"type": "function", "function": convert_to_cpp_function},
    {"type": "function", "function": save_cpp_to_file_function},
    {"type": "function", "function": get_system_info_function},
    {"type": "function", "function": compile_and_run_function},
]

In [26]:
# ── Tool dispatcher (mirrors handle_tool_call from the example flow) ──────────

def handle_tool_call(message) -> dict:
    """Route a single tool call from the LLM to the correct Python function
    and return a 'tool' role message ready to append to the conversation."""
    tool_call  = message.tool_calls[0]
    arguments  = json.loads(tool_call.function.arguments)
    name       = tool_call.function.name

    if name == "convert_to_cpp":
        content = convert_to_cpp(arguments["python_code"])

    elif name == "save_cpp_to_file":
        content = save_cpp_to_file(
            arguments["cpp_code"],
            arguments.get("filename", "main.cpp"),
        )

    elif name == "get_system_info":
        content = get_system_info()

    elif name == "compile_and_run":
        content = compile_and_run(
            arguments["compile_command"],
            arguments["run_command"],
        )

    else:
        content = f"Unknown tool: {name}"

    return {
        "role":         "tool",
        "content":      content,
        "tool_call_id": tool_call.id,
    }

In [ ]:
# ── System message ─────────────────────────────────────────────────────────────

SYSTEM_MESSAGE = """
You are an expert Python-to-C++ conversion agent. When the user provides Python code, follow these steps in order — do not skip any:

1. Call convert_to_cpp with the Python code to generate high-performance C++.
2. Call save_cpp_to_file with the returned C++ code to persist it to main.cpp.
3. Call get_system_info to learn about the machine's OS, CPU, and compiler.
4. Call compile_and_run with the compile and run commands derived from the system info.
   Use the fastest flags appropriate for the system (e.g., -Ofast -march=native -flto -DNDEBUG for g++).
5. Report back to the user with:
   - The generated C++ code (in a code block)
   - The file path where it was saved
   - The program's execution output
""".strip()


# ── Main chat function (tool-call loop, same pattern as the example) ──────────

def chat(message: str, history: list) -> str:
    """Orchestrate the full Python → C++ pipeline using chained tool calls.

    Each tool's output feeds into the LLM's next decision, forming the chain:
      convert_to_cpp → save_cpp_to_file → get_system_info → compile_and_run → reply
    """
    # Convert Gradio history list to OpenAI message dicts
    history_msgs = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = (
        [{"role": "system", "content": SYSTEM_MESSAGE}]
        + history_msgs
        + [{"role": "user", "content": message}]
    )

    response = llm_client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages,
        tools=tools,
    )

    # Loop: keep handling tool calls until the model is done
    # (one tool output becomes context for the next tool decision)
    while response.choices[0].finish_reason == "tool_calls":
        tool_message   = response.choices[0].message
        tool_response  = handle_tool_call(tool_message)

        # Append both the assistant's tool-call request and the tool result
        messages.append(tool_message)
        messages.append(tool_response)

        response = llm_client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=messages,
            tools=tools,
        )

    return response.choices[0].message.content


# ── Gradio UI ──────────────────────────────────────────────────────────────────

gr.ChatInterface(
    fn=chat,
    title="Python → C++ Converter Agent",
    description=(
        "Paste Python code and the agent will:\n"
        "1. Convert it to high-performance C++\n"
        "2. Save it to `main.cpp`\n"
        "3. Detect your system's compiler\n"
        "4. Compile & run it — then report the output"
    ),
   
).launch()